<a href="https://colab.research.google.com/github/Miguelapp10/Google_Colab_GCP/blob/main/Postventa_Pendiente_devolucion_tienda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install google-colab google-cloud-bigquery
!pip install xlsxwriter
#!pip install pyautogui
#!pip install smtplib
!pip install Openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
from google.colab import files

auth.authenticate_user()

import gspread as gs
import gspread_dataframe as gd
from google.auth import default
creds, _ = default()

gc = gs.authorize(creds)

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from datetime import datetime, timedelta,timezone

# Ingresa el ID de tu proyecto de Google Cloud
project_id = 'bi-fcom-drmb-local-pe-sbx'

# Configura la conexión a BigQuery
client = bigquery.Client(project=project_id)

# Ejemplo de consulta
query = """
SELECT  *  FROM `bi-fcom-drmb-local-pe-sbx.Pidgeotto_Devolucion.Devolucion_Tienda_Backstore` where Fecha_Entregado_Tienda>='2024-01-01'
"""
# Ejecutar la consulta
result = client.query(query)

# Convertir los resultados en un DataFrame de Pandas
Backstore = result.to_dataframe()

In [ ]:
## BU_Negocio
#Filter_Backstore= Backstore[Backstore['BU_Devolucion'] == "TOTTUS"]

## BU_Negocio
Filter_Backstore = Backstore[Backstore['ESTADO_ACTUAL'] == "SOLICITUD DE SERVICIO"]

## BU_Negocio
Filter_Backstore = Filter_Backstore[Filter_Backstore['Estado_Backstore'] != "ANNULLED"]
## BU_Negocio
#Filter_Backstore = Filter_Backstore[Filter_Backstore['Tienda'] != "Falabella.com Lince"]

## BU_Negocio
Filter_Backstore =Filter_Backstore[ (Filter_Backstore['ST'].apply(lambda x: pd.isna(x) or x == '' or x == '0'))]

 ##Assuming 'Fecha_Recepcionado_Tienda' is in datetime format
Filter_Backstore['Año_mes'] = Filter_Backstore['Fecha_Entregado_Tienda_v2'].dt.strftime('%Y-%m')

# Suponiendo que Filter_Backstore es tu DataFrame
Filter_Backstore['Precio'] = pd.to_numeric(Filter_Backstore['Precio'])

# Create a pivot table
pivot_table_1 = pd.pivot_table(Filter_Backstore,
                             values=['RLO'],
                             index=['Status_Buffer'],
                             columns=['Año_mes'],
                             aggfunc={'RLO': 'count'},
                             margins=True,
                             margins_name='Total'
                             ).rename(columns={'RLO': 'Cantidad'}
                                      ).rename_axis(columns={'Status_Buffer': 'Estado_Buffer'})

# Format the 'Monto' column with a currency symbol using applymap
#pivot_table['Monto'] = pivot_table['Monto'].applymap('S/.{:,.2f}'.format)
# Rellenar los valores NaN con ''
pivot_table_1.fillna('', inplace=True)

# Create HTML representation of the styled pivot table
html_table_1 = pivot_table_1.to_html(classes='styled-table', escape=False)

# Print the head of the pivot table
print(pivot_table_1.head())

              Cantidad                      
Año_mes        2024-01 2024-02 2024-03 Total
Status_Buffer                               
No                24.0    43.0            67
Si                              1319.0  1319
Total             24.0    43.0  1319.0  1386


In [ ]:
# Function to categorize the difference
def categorize_difference(days):
    if days <= 7:
        return "1_Sem"
    elif days <= 14:
        return "2_Sem"
    elif days <= 21:
        return "3_Sem"
    elif days <= 28:
        return "4_Sem"
    else:
        return "4_Sem_+"

# Apply the function to create the new column
Filter_Backstore['Rango_Fecha_Tienda_'] = Filter_Backstore['Dias_en_tiendas'].apply(categorize_difference)

# Create a pivot table
pivot_table_2 = pd.pivot_table(Filter_Backstore,
                             values=['RLO'],
                             index=['Tienda'],
                             columns=['Rango_Fecha_Tienda_'],
                             aggfunc={'RLO': 'count'},
                              margins=False,
                             margins_name='Total'
                             ).rename(columns={'RLO': 'Cantidad'}
                                      ).rename_axis(columns={'Tienda': 'Tienda'})
# Format the 'Monto' column with a currency symbol using applymap
#pivot_table['Monto'] = pivot_table['Monto'].applymap('S/.{:,.2f}'.format)

# Calculate the 'Total' column by summing across all columns
pivot_table_2['Total'] = pivot_table_2.sum(axis=1)

# Sort the pivot table by the 'Total' column in descending order
pivot_table_2_sorted = pivot_table_2.sort_values(by='Total', ascending=False)

# Rellenar los valores NaN con ''
pivot_table_2_sorted.fillna('', inplace=True)
# Create HTML representation of the styled pivot table
html_table_2 = pivot_table_2_sorted.to_html(classes='styled-table', escape=False)

# Print the head of the pivot table
print(pivot_table_2_sorted.head())

                              Cantidad                            Total
Rango_Fecha_Tienda_              1_Sem 2_Sem 3_Sem 4_Sem 4_Sem_+       
Tienda                                                                 
FALABELLA MALL DEL SUR           141.0  87.0                 1.0  229.0
FALABELLA MALL PLAZA TRUJILLO    103.0  19.0         2.0     2.0  126.0
FALABELLA AREQUIPA CAYMA          49.0   3.0   6.0                 58.0
FALABELLA SAN ISIDRO              56.0   2.0                       58.0
FALABELLA SANTA ANITA             39.0  10.0                 1.0   50.0


In [ ]:
# Create a pivot table

pivot_table_5 = Filter_Backstore.groupby(['Rango_Fecha_Tienda_']).agg(
    Monto=pd.NamedAgg(column="Precio", aggfunc="sum"))
# Format the 'Monto' column with a currency symbol using applymap
#pivot_table['Monto'] = pivot_table['Monto'].applymap('S/.{:,.2f}'.format)

# Calculate the 'Total' column by summing across all columns
#pivot_table_5['Total'] = pivot_table_5.sum(axis=1)

# Sort the pivot table by the 'Total' column in descending order
#pivot_table_5_sorted = pivot_table_5.sort_values(by='Total', ascending=False)

# Rellenar los valores NaN con ''
pivot_table_5.fillna('', inplace=True)
# Create HTML representation of the styled pivot table
html_table_5 = pivot_table_5.to_html(classes='styled-table', escape=False)

# Print the head of the pivot table
print(pivot_table_5.head())

                         Monto
Rango_Fecha_Tienda_           
1_Sem                180910.49
2_Sem                 60129.11
3_Sem                  4634.20
4_Sem                  1249.70
4_Sem_+                6939.00


In [ ]:
# Connect sheet
sheet = gc.open_by_key('1MVHS3J_2v0haVS_MGGSOhfw2_0y07B9cOuZjNNKFxmI')
worksheet = sheet.worksheet('Devolución_tienda')
# Get all values from the worksheet
data = worksheet.get_all_values()
# Convert the data to a Pandas DataFrame
offline = pd.DataFrame(data[1:], columns=data[0])

offline = offline[['RASTREO','RASTREO2','DESCRICION POR ENTREGAR','Tienda','Orden Seller','SKU','valor_sku','Fecha solicitud',
                   'Guía', 'OPL','Estado de Recoleccion','Día visita tienda','Fecha de Recoleccion','Estado de Entrega','Fecha de entrega',
                   'Comentario Postventa','NEGOCIO']]

# Cambiar el nombre de la columna
offline = offline.rename(columns={'DESCRICION POR ENTREGAR':'DESCRIPCION','Tienda': 'TIENDA','Orden Seller':'ORDEN','valor_sku':'PRECIO','Fecha solicitud':'FECHA_SOLICITUD',
                                  'Guía':'GUIA','Estado de Recoleccion':'ESTADO_RECOLECCION','Día visita tienda':'DIA_RECOLECCION','Fecha de Recoleccion':'FECHA_RECOLECCION',
                                  'Estado de Entrega':'ESTADO_ENTREGA','Fecha de entrega':'FECHA_ENTREGA','Comentario Postventa':'COMENTARIO_POSTVENTA'})
# Filtrar el DataFrame por la columna 'ESTADO_RECOLECCION'
offline = offline[(offline['ESTADO_RECOLECCION'].str.contains('SOLICITUD DE SERVICIO', case=False))]
offline =offline[ (offline['COMENTARIO_POSTVENTA'].apply(lambda x: pd.isna(x) or x == '' or x == '0'))]

# Convert it to datetime first
offline['FECHA_SOLICITUD'] = pd.to_datetime(offline['FECHA_SOLICITUD'])
# Convert 'FECHA_SOLICITUD' column to datetime format
offline['FECHA_SOLICITUD'] = pd.to_datetime(offline['FECHA_SOLICITUD'], format='%d/%m/%Y')

## Fecha 2024-01-01
filter_offline = offline[offline['FECHA_SOLICITUD'] >= pd.Timestamp(2024, 1, 1)]

## BU_Negocio
##filter_offline = offline_24[offline_24['NEGOCIO'] == "Tottus"]

# Calculate the difference between the current date and Fecha_Entregado_Tienda_v2
filter_offline['DIA_TIENDA'] = (datetime.now() - filter_offline['FECHA_SOLICITUD']).dt.days

# Conditionally assign values based on the calculated difference
filter_offline['Status_Buffer'] = 'Si'
filter_offline.loc[filter_offline['DIA_TIENDA'] > 15, 'Status_Buffer'] = 'No'

 ##Assuming 'Fecha_Recepcionado_Tienda' is in datetime format
filter_offline['Año_mes'] = filter_offline['FECHA_SOLICITUD'].dt.strftime('%Y-%m')


# Apply the function to create the new column
filter_offline['Rango_Fecha_Tienda'] = filter_offline['DIA_TIENDA'].apply(categorize_difference)

# Convertir las columnas a mayúsculas
filter_offline['TIENDA'] = filter_offline['TIENDA'].str.strip().str.upper()

# Suponiendo que Filter_Backstore es tu DataFrame
filter_offline['PRECIO'] = pd.to_numeric(filter_offline['PRECIO'])

"""print(filter_offline)
info = info[['TIENDA','RLO','GUIA']]"""

<ipython-input-7-b85c4fac8c8c>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_offline['DIA_TIENDA'] = (datetime.now() - filter_offline['FECHA_SOLICITUD']).dt.days
<ipython-input-7-b85c4fac8c8c>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_offline['Status_Buffer'] = 'Si'
<ipython-input-7-b85c4fac8c8c>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

"print(filter_offline)\ninfo = info[['TIENDA','RLO','GUIA']]"

In [ ]:
# Create a pivot table
pivot_table_3 = pd.pivot_table(filter_offline,
                             values=['RASTREO'],
                             index=['Status_Buffer'],
                             columns=['Año_mes'],
                             aggfunc={'RASTREO': 'count'},
                             margins=True,
                             margins_name='Total'
                             ).rename(columns={'RASTREO': 'Cantidad'}
                                      ).rename_axis(columns={'Status_Buffer': 'Estado_Buffer'})

# Format the 'Monto' column with a currency symbol using applymap
#pivot_table['Monto'] = pivot_table['Monto'].applymap('S/.{:,.2f}'.format)
# Rellenar los valores NaN con ''
pivot_table_3.fillna('', inplace=True)

# Create HTML representation of the styled pivot table
html_table_3 = pivot_table_3.to_html(classes='styled-table', escape=False)

# Print the head of the pivot table
print(pivot_table_3.head())

              Cantidad                      
Año_mes        2024-01 2024-02 2024-03 Total
Status_Buffer                               
No                 7.0    23.0            30
Si                               184.0   184
Total              7.0    23.0   184.0   214


In [ ]:
# Create a pivot table
pivot_table_4 = pd.pivot_table(filter_offline,
                             values=['RASTREO'],
                             index=['TIENDA'],
                             columns=['Rango_Fecha_Tienda'],
                             aggfunc={'RASTREO': 'count'},
                              margins=False,
                             margins_name='Total'
                             ).rename(columns={'RASTREO': 'Cantidad'}
                                      ).rename_axis(columns={'TIENDA': 'TIENDA'})
# Format the 'Monto' column with a currency symbol using applymap
#pivot_table['Monto'] = pivot_table['Monto'].applymap('S/.{:,.2f}'.format)


# Calculate the 'Total' column by summing across all columns
pivot_table_4['Total'] = pivot_table_4.sum(axis=1)

# Sort the pivot table by the 'Total' column in descending order
pivot_table_4_sorted = pivot_table_4.sort_values(by='Total', ascending=False)

# Rellenar los valores NaN con ''
pivot_table_4_sorted.fillna('', inplace=True)
# Create HTML representation of the styled pivot table
html_table_4 = pivot_table_4_sorted.to_html(classes='styled-table', escape=False)


# Print the head of the pivot table
print(pivot_table_4_sorted.head())

                         Cantidad                           Total
Rango_Fecha_Tienda          1_Sem 2_Sem 3_Sem 4_Sem 4_Sem_+      
TIENDA                                                           
FALABELLA AREQUIPA CAYMA      8.0  10.0   5.0                23.0
FALABELLA MALL DEL SUR       13.0   3.0                      16.0
FALABELLA ANGAMOS            11.0   3.0                      14.0
FALABELLA MIRAFLORES         11.0   2.0                      13.0
FALABELLA CAJAMARCA           3.0   5.0   3.0                11.0


In [ ]:
# Create a pivot table

pivot_table_6 = filter_offline.groupby(['Rango_Fecha_Tienda']).agg(
    Monto=pd.NamedAgg(column="PRECIO", aggfunc="sum"))
# Format the 'Monto' column with a currency symbol using applymap
#pivot_table['Monto'] = pivot_table['Monto'].applymap('S/.{:,.2f}'.format)

# Calculate the 'Total' column by summing across all columns
#pivot_table_5['Total'] = pivot_table_5.sum(axis=1)

# Sort the pivot table by the 'Total' column in descending order
#pivot_table_5_sorted = pivot_table_5.sort_values(by='Total', ascending=False)

# Rellenar los valores NaN con ''
pivot_table_6.fillna('', inplace=True)
# Create HTML representation of the styled pivot table
html_table_6 = pivot_table_6.to_html(classes='styled-table', escape=False)

# Print the head of the pivot table
print(pivot_table_6.head())

                       Monto
Rango_Fecha_Tienda          
1_Sem               31892.47
2_Sem                5421.82
3_Sem                1328.32
4_Sem                 498.80
4_Sem_+              5297.30


In [ ]:
Filter_Backstore= Filter_Backstore[['Fecha_Entregado_Tienda_v2','Fecha_Confirmacion_Envio','T_Dia_EntreTienda_ConfiEnvio','Tienda',
                                             'submotivo_traductor','Estado_RLO','Estado_3PL','Estado_Backstore','ESTADO_ACTUAL',
                                        'Status_Buffer','OC','RLO','Etiqueta_Pre_Impresa','GUIA','Product_Name','Precio','ST','Rango_Fecha_Tienda','Dias_en_tiendas']]
filter_offline=filter_offline[['RASTREO','DESCRIPCION','TIENDA','ORDEN','SKU','PRECIO','FECHA_SOLICITUD','GUIA','OPL','DIA_RECOLECCION',
                                     'ESTADO_RECOLECCION','FECHA_RECOLECCION','ESTADO_ENTREGA','FECHA_ENTREGA','NEGOCIO','Status_Buffer','Rango_Fecha_Tienda','DIA_TIENDA']]

In [ ]:
import time
import datetime
##import pyautogui
import smtplib
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import math  # Necesario para verificar NaN
import os

# Obtener la fecha actual y calcular la semana correspondiente
today = datetime.date.today()
week_number = today.strftime("%V")

# Definir el nombre del archivo Excel con el formato deseado
nombre_archivo = f"Devoluciones_Pendientes_W{week_number}.xlsx"
# Escribir los DataFrames en un archivo Excel
with pd.ExcelWriter(nombre_archivo) as writer:
    # Escribir 'filtered_rlos_tienda' en una hoja llamada 'filtered_rlos_tienda'
    Filter_Backstore.to_excel(writer, sheet_name='Backstore', index=False)

    # Escribir 'filter_offline' en una hoja llamada 'filter_offline'
    filter_offline.to_excel(writer, sheet_name='Offline', index=False)
# Obtener el libro de trabajo y las hojas
    workbook = writer.book
    backstore_sheet = writer.sheets['Backstore']
    offline_sheet = writer.sheets['Offline']

    # Formato para los títulos de fondo de color naranja
    header_format = workbook.add_format({'bg_color': '#FFA500', 'bold': True})

    # Aplicar formato a los títulos en Backstore
    for col_num, value in enumerate(Filter_Backstore.columns.values):
        backstore_sheet.write(0, col_num, value, header_format)

    # Aplicar filtros en Backstore
    backstore_sheet.autofilter(0, 0, len(Filter_Backstore.index), len(Filter_Backstore.columns) - 1)

    # Establecer el ancho de la columna en 30 en Backstore
    backstore_sheet.set_column(0, len(Filter_Backstore.columns) - 1, 30)

    # Aplicar formato a los títulos en Offline
    for col_num, value in enumerate(filter_offline.columns.values):
        offline_sheet.write(0, col_num, value, header_format)

    # Aplicar filtros en Offline
    offline_sheet.autofilter(0, 0, len(filter_offline.index), len(filter_offline.columns) - 1)

    # Establecer el ancho de la columna en 30 en Offline
    offline_sheet.set_column(0, len(filter_offline.columns) - 1, 30)
# Calcular el total de devoluciones en los DataFrames
total_devoluciones_Backstore = Filter_Backstore.shape[0]
total_devoluciones_Offline = filter_offline.shape[0]

# Calcular la suma de precios en los DataFrames
suma_precios_Backstore = Filter_Backstore['Precio'].sum()
suma_precios_Offline = filter_offline['PRECIO'].sum()

# Configuración
email_usuario = 'mpazos@falabella.com'
contrasena = 'Mapp1234%'
asunto_base = f"DETALLE DE DEVOLUCIONES PENDIENTES {today.year} - WEEK {week_number} - {today}"

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(email_usuario, contrasena)

# Configurar el mensaje
msg = MIMEMultipart()
msg['From'] = email_usuario
msg['To'] = 'ypalominop@falabella.com,mlaurente@falabella.com'
msg['Subject'] = asunto_base
msg['CC'] = 'lshuan@falabella.com,mfmartinez@falabella.com'

# Darle un nivel de importancia al correo electrónico (en este caso, Alto)
msg.add_header('Importance', 'High')

# Crear el cuerpo del correo en formato HTML
mensaje_html = f"""
<html>
  <head>
    <style>
      .styled-table th {{
        background-color: orange;
        font-weight: bold;
      }}
    </style>
  </head>
  <body>
    <p>Estimado Mauricio!</p>
    <p>Informo que tenemos {total_devoluciones_Backstore} devoluciones pendientes en backstore con un valorizado S/.{suma_precios_Backstore} y
     {total_devoluciones_Offline} devoluciones pendientes en offline con una valorizado S/.{suma_precios_Offline} .</p>
    <p>Comparto el detalle de las devoluciones pendiente de Tottus hasta el día de hoy.</p>
    <p>Por favor, su apoyo trasladando los pendientes a Plataforma para su pronto despacho.</p>
    <p>Además, adjunto un archivo en la cual detalla las devoluciones que se encuentran fuera de buffer y no han tenido salida por los dos flujos.</p>

    <p><span style="font-size: 16px"><u><strong> Estado de Buffer de Backstore</strong></u></span><br></p>
    {html_table_1}
    <p><span style="font-size: 16px"><u><strong>Detalle por Tienda con mas Devoluciones Pendientes Backstore.</strong></u></span><br></p>
    {html_table_2}
    <p><span style="font-size: 16px"><u><strong>Monto Total de Devoluciones Pendientes Backstore.</strong></u></span><br></p>
    {html_table_5}
    <p><span style="font-size: 16px"><u><strong>Estado de Buffer de Offline.</strong></u></span><br></p></p>
    {html_table_3}
    <p><span style="font-size: 16px"><u><strong>Detalle por Tienda con mas Devoluciones Pendientes Offline.</strong></u></span><br></p></p>
    {html_table_4}
    <p><span style="font-size: 16px"><u><strong>Monto Total de Devoluciones Pendientes Offline.</strong></u></span><br></p>
    {html_table_6}

  </body>
</html>
"""

# Adjuntar el cuerpo del correo en formato HTML
msg.attach(MIMEText(mensaje_html, 'html'))

# Adjuntar el archivo Excel al correo electrónico
filename = nombre_archivo
attachment = open(nombre_archivo, "rb")

part = MIMEBase('application', 'octet-stream')
part.set_payload(attachment.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

msg.attach(part)

# Enviar el correo electrónico
server.sendmail(email_usuario,msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')